# Pulsar Star Prediction

<img width="1134" alt="methods" src="https://user-images.githubusercontent.com/28593767/112789439-0cda6480-9098-11eb-96e5-9fc335f3bfd8.png">

In [1]:
import numpy as np
import csv

np.random.seed(222)

RND_MEAN = 0
RND_STD = 0.003

LEARNING_RATE = 0.003

In [2]:
def binary_classification_exec(epoch_count=10, mb_size=10, report=1,train_rate = 0.8): 
    binary_load_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report, train_rate)

In [3]:
# 오직 실수 데이터만 존재하므로 원-핫 벡터 과정은 생략 가능하다.
# 데이터 로드 및 처리 함수
def binary_load_dataset():
    with open('data_nn/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt 
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32') # Numpy 배열 구조로 변환

In [4]:
# 파라미터 초기화 함수
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [5]:
def train_and_test(epoch_count, mb_size, report, train_rate): 
    step_count = arrange_data(mb_size, train_rate)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count): 
        losses, accs = [], []
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n) 
            loss, acc = run_train(train_x, train_y) 
            losses.append(loss)
            accs.append(acc)
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch + 1, np.mean(losses), np.mean(accs), acc))
        
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

In [6]:
def arrange_data(mb_size, train_rate):
    global data, shuffle_map, test_begin_idx
    # 주의!!! Numpy의 메소드는 np.arange() 이다 (Arrange가 아니다!)
    shuffle_map = np.arange(data.shape[0]) 
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * train_rate) // mb_size
    test_begin_idx = step_count * mb_size
    
    return step_count

In [7]:
def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt 
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

In [18]:
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt 
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size * nth : mb_size * (nth + 1)]] 
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [19]:
def run_train(x, y):
    # 순전파 과정
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    # 역전파 과정
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp) 
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

In [20]:
def run_test(x, y):
    output, _ = forward_neuralnet(x) 
    accuracy = eval_accuracy(output, y) 
    return accuracy

In [21]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias 
    return output, x

In [22]:
def backprop_neuralnet(G_output, x): 
    global weight, bias
    g_output_w = x.transpose()
    G_w = np.matmul(g_output_w, G_output) 
    G_b = np.sum(G_output, axis=0)
    
    weight -= LEARNING_RATE * G_w 
    bias -= LEARNING_RATE * G_b

In [23]:
def forward_postproc(output, y):
    CEE = sigmoid_cross_entropy_with_logits(y,output)  # 시그모이드 교차 엔트로피 함수
    loss = np.mean(CEE)
    
    return loss, [y, output, CEE]

In [24]:
def backprop_postproc(G_loss, aux):
    y, output, CEE = aux
    G_loss = 1.0

    g_loss_entropy = 1.0 / np.prod(CEE.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y,output)

    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy

    return G_output

In [25]:
def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))

def relu(x):
    return np.maximum(x, 0)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [26]:
def eval_accuracy(output,y):
    estimate = np.greater(output,0)
    answer = np.greater(y,0.5)
    correct = np.equal(estimate, answer)
    return np.mean(correct)

In [27]:
binary_classification_exec()

Epoch 1: loss=0.376, accuracy=0.951/0.964
Epoch 2: loss=0.344, accuracy=0.958/0.947
Epoch 3: loss=0.365, accuracy=0.958/0.972
Epoch 4: loss=0.327, accuracy=0.960/0.972
Epoch 5: loss=0.338, accuracy=0.961/0.973
Epoch 6: loss=0.359, accuracy=0.957/0.972
Epoch 7: loss=0.347, accuracy=0.960/0.971
Epoch 8: loss=0.317, accuracy=0.962/0.973
Epoch 9: loss=0.353, accuracy=0.961/0.969
Epoch 10: loss=0.348, accuracy=0.961/0.966

Final Test: final accuracy = 0.966


# 정확도 계산 함수 정의
결과값을 보면 학습 초기부터 정확도가 "너무" 높다.

그 이유는 데이터의 분포를 보면 알 수 있는데, 데이터의 타입이 0이 16259개, 1이 1639개로 일반 별의 분포가 90%를 차지하고 있다는 점이다.

따라서 정확도를 측정하는 과정에서 문제가 발생한다.

* 이러한 착시 현상으로 인해 정확도가 정확하게 보여지지 않는 상황을 타개하기 위해 신경망의 성능을 더욱 잘 보여줄 수 있는 또 다른 평가 지표가 필요하다.
* 대표적인 지표에는 **정밀도(precision)** 와 **재현율(recall)** 이 있다.
    + 정밀도는 신경망이 참으로 예측한 것 가운데 정답이 참인 비율을 의미한다. 
        - TP / (TP + FP)
    + 재현율은 거꾸로 정답이 참인 것들 가운데 신경망이 참으로 예측한 비율을 의미한다.
        - TP / (TP + FN)
    
> TP : 신경망의 추측이 '참(P)'이며, 데이터의 정답 또한 '참'(T)으로 정확하게 평가한 결과 (TRUE POSITIVE)
>
> TN : 신경망의 추측이 '거짓(N)'이며, 데이터의 정답은 '참'(T)으로 부정확하게 평가한 결과 (TRUE NEGATIVE)
>
> FP : 신경망의 추측이 '참(P)'이며, 데이터의 정답은 '거짓(F)'으로 부정확하게 평가한 결과 (FALSE POSITIVE)
>
> FN : 신경망의 추측이 '거짓(N)'이며, 데이터의 정답 또한 '거짓(F)'으로 정확하게 평가한 결과 (FALSE NEGATIVE)

정밀도와 재현율의 조화 평균인 **F1 score** 가 정확도 대신 사용되기도 한다.

조화 평균은 *역수의 차원에서 평균을 구하고 다시 역수*를 취해 원래 차원의 값으로 돌아오게 하여 구할 수 있다.

![f1](https://user-images.githubusercontent.com/28593767/112940203-ca805880-9167-11eb-9ad1-895a6674cb67.png)